In [103]:
import os
import pyspark 
from pyspark.sql import SparkSession
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [104]:
from dotenv import load_dotenv
load_dotenv()
key_filepath = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

In [105]:
spark = SparkSession.builder \
    .master("local[3]") \
    .appName("Airbnb Listings") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile",key_filepath  ) \
    .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .getOrCreate()       


    

In [106]:

# Read data from GCS
df = spark.read.parquet("gs://airbnb-listings-421017-bucket/airbnb_listings.parquet")


In [107]:
df.show(5)

+-------+--------------------+--------------------+-------------------+------------------+--------------------+--------------------+----------------+---------------------+----------------------+--------+---------+----------+------------------+--------------------+-------+--------------------+-------------+---------------+------------+---------+--------+----+-----------------+--------------------+-------------------+-----------------+-----+-----------------+
|     id|                name|             summary|          longitude|          latitude|               space|         description|instant_bookable|neighborhood_overview|neighbourhood_cleansed| host_id|host_name|host_since|host_response_time|              street|zipcode|review_scores_rating|property_type|      room_type|accommodates|bathrooms|bedrooms|beds|reviews_per_month|           amenities|cancellation_policy|number_of_reviews|price|__index_level_0__|
+-------+--------------------+--------------------+-------------------+-----

In [108]:
# Check if df is an RDD
if isinstance(df, pyspark.rdd.RDD):
    print("df is an RDD")
else:
    print("df is not an RDD")


df is not an RDD


In [ ]:
def parse_point(row, column_names):
  features = [row[col] for col in column_names]
  label = row[-1]
  return (features, label)

# Get column names (assuming they are known beforehand)
column_names = df.columns[1:]  
print(column_names)
# Apply parse_point with column names
parsed_data = parsed_data_rdd.map(lambda row: parse_point(row, column_names))
print(parsed_data.collect()[0])

In [109]:

from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, FloatType

def parse_point(row, column_names, categorical_cols):
    """
    Parses a row into features (numerical) and label.
    Handles categorical column conversion using one-hot encoding.

    Args:
        row (Spark Row): Row from the DataFrame.
        column_names (list): List of column names.
        categorical_cols (list): List of categorical column names.

    Returns:
        tuple: (features, label) where features is a NumPy array and label is a float.
    """

    features = [row[col] for col in column_names if col not in categorical_cols]

    # One-hot encode categorical columns
    from pyspark.ml.feature import OneHotEncoderEstimator
    ohe = OneHotEncoderEstimator(inputCols=categorical_cols, outputCols=["encoded_" + col for col in categorical_cols])
    encoded_features = ohe.transform(row.toDf()).select("encoded_*").rdd.flatMap(lambda r: r.asDict(True).values()).collect()[0]

    features.extend(encoded_features)

    label = row[-1]  # Assuming label is the last column
    return np.array(features, dtype=float), float(label)

# Get column names
column_names = df.columns
# # Identify categorical columns (replace with your actual list)
categorical_cols = []  
idx=0
for feat in df.rdd.collect()[0]:
#     print(feat)
    if isinstance(feat, str):
        categorical_cols.append(column_names[idx])   
    idx+=1


# Define UDF (user-defined function) for parsing with Spark SQL functions
parse_point_udf = udf(lambda row: parse_point(row, column_names, categorical_cols), ArrayType(FloatType()))

# Apply the UDF to the DataFrame
parsed_data = df.rdd.map(parse_point_udf)

# Example: Print the first parsed data point (features and label)
print(parsed_data.collect()[0])


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 70.0 failed 1 times, most recent failure: Lost task 0.0 in stage 70.0 (TID 70) (Lenovo-Ideapad-520 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "E:\spark-3.5.1-bin-hadoop3\python\lib\pyspark.zip\pyspark\worker.py", line 1247, in main
  File "E:\spark-3.5.1-bin-hadoop3\python\lib\pyspark.zip\pyspark\worker.py", line 1239, in process
  File "E:\spark-3.5.1-bin-hadoop3\python\lib\pyspark.zip\pyspark\serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "E:\spark-3.5.1-bin-hadoop3\python\lib\pyspark.zip\pyspark\util.py", line 83, in wrapper
    return f(*args, **kwargs)
  File "C:\Users\LP-7263\AppData\Roaming\Python\Python310\site-packages\pyspark\sql\udf.py", line 423, in wrapper
    return self(*arg

In [78]:
import numpy as np
# Initialize coefficients (weights)
# print(parsed_data.first()[0])
num_features = len(parsed_data.first()[0])
print("num_features: "+ str(num_features))
weights = np.random.rand(num_features)

num_features: 28


In [79]:
# Define the Hypothesis Function
def hypothesis(x, weights):
    print(x)
    return np.dot(x, weights)

In [80]:
# Define the Cost Function
def cost_function(data, weights):
    n = data.count()
    squared_errors = data.map(lambda point: (hypothesis(point[0], weights) - point[1]) ** 2)
    return squared_errors.reduce(lambda x, y: x + y) / (2 * n)


In [81]:
# Gradient Descent
def gradient_descent(data, weights, learning_rate):
    n = data.count()
    gradients = data.map(lambda point: np.multiply(hypothesis(point[0], weights) - point[1], point[0]))
    print(gradients)
    gradient_sum = gradients.reduce(lambda x, y: np.add(x, y))
    return np.subtract(weights, np.multiply(learning_rate / n, gradient_sum))


In [82]:
# Hyperparameters
learning_rate = 0.01
num_iterations = 100

In [83]:
# Training loop
train_no=1
# Check if df is an RDD
if isinstance(parsed_data, pyspark.rdd.RDD):
    print("df is an RDD")
else:
    print("df is not an RDD")
print(parsed_data)
for _ in range(num_iterations):
    print("train no: "+str(train_no))
    weights = gradient_descent(parsed_data, weights, learning_rate)
    cost = cost_function(parsed_data, weights)
    train_no+=1
    print("Cost:", cost)


df is an RDD
PythonRDD[24] at collect at C:\Users\LP-7263\AppData\Local\Temp\ipykernel_17824\3913970460.py:11
train no: 1
PythonRDD[55] at RDD at PythonRDD.scala:53


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 51.0 failed 1 times, most recent failure: Lost task 0.0 in stage 51.0 (TID 51) (Lenovo-Ideapad-520 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "E:\spark-3.5.1-bin-hadoop3\python\lib\pyspark.zip\pyspark\worker.py", line 1247, in main
  File "E:\spark-3.5.1-bin-hadoop3\python\lib\pyspark.zip\pyspark\worker.py", line 1239, in process
  File "E:\spark-3.5.1-bin-hadoop3\python\lib\pyspark.zip\pyspark\serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\Users\LP-7263\AppData\Roaming\Python\Python310\site-packages\pyspark\rdd.py", line 1919, in func
    initial = next(iterator)
  File "E:\spark-3.5.1-bin-hadoop3\python\lib\pyspark.zip\pyspark\util.py", line 83, in wrapper
    return f(*args, **kwarg

In [ ]:
# Stop SparkSession
spark.stop()

In [20]:
import sys

print(sys.executable)


C:\ProgramData\anaconda3\python.exe
